In [1]:
import numpy as np
import pandas as pd
import time
import datetime
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from wordcloud import WordCloud, STOPWORDS 
import seaborn as sns
from nltk.corpus import stopwords

C:\Users\sony\Anaconda3\envs\NLP\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\sony\Anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance:

In [3]:
df = pd.read_csv("mabel.txt", header=None, encoding='utf-8', error_bad_lines=False)


b'Skipping line 23: expected 2 fields, saw 6\nSkipping line 44: expected 2 fields, saw 3\n'


In [4]:
df= df.drop(0)
df.columns=['Date', 'Chat']
Message= df["Chat"].str.split("-", n = 1, expand = True) 
df['Date']=df['Date'].str.replace(",","") 
df['Time']=Message[0]
df['Text']=Message[1]
Message1= df["Text"].str.split(":", n = 1, expand = True) 
df['Text']=Message1[1]
df['Name']=Message1[0]
df=df.drop(columns=['Chat'])
df['Text']=df['Text'].str.lower()
df=df.dropna()

In [5]:
dataset=df
dataset

,Date,Time,Text,Name
1,05/12/19,1:42 pm,hi this is mabel we just spoke,Mabel Infoziant
2,05/12/19,1:42 pm,what’s your full name,Mabel Infoziant
3,05/12/19,1:42 pm,ramisha rani k,AR❤
4,05/12/19,1:42 pm,ok,Mabel Infoziant
5,05/12/19,1:42 pm,ramisharanik@gmail.com,Mabel Infoziant
6,05/12/19,1:43 pm,your email id?,Mabel Infoziant
7,05/12/19,1:43 pm,yes mam,AR❤
8,05/12/19,1:43 pm,i will send 2 abstracts for u to start working,Mabel Infoziant
9,05/12/19,1:43 pm,yeah mam,AR❤
10,05/12/19,1:43 pm,give me the list that u have too,Mabel Infoziant


In [6]:
def flirt_analysis(dataset):
    flirt_words = ['Sweetheart','kiss', 'hug', 'date', 'cute', 'beautiful', 'sexy', 'hot', 'adorable', 'darling',
                   'love','romantic']

    result = []
    Talker_chat = dataset[dataset['Name'] == dataset['Name'].value_counts().idxmax()]
    Less_chat = dataset[dataset['Name'] == dataset['Name'].value_counts().idxmin()]

    unique_Frequency_Talker = Talker_chat['Text'].str.split(' ', expand=True).stack().value_counts()
    unique_Frequency_Lesser = Less_chat['Text'].str.split(' ', expand=True).stack().value_counts()

    Talker_Filter_list = unique_Frequency_Talker[unique_Frequency_Talker.index.isin(flirt_words)]
    Less_Filter_list = unique_Frequency_Lesser[unique_Frequency_Lesser.index.isin(flirt_words)]

    if not Talker_Filter_list.empty:
        Talker_Filter_list['Flirt_Frequency'] = (Talker_Filter_list / len(flirt_words)) * 100
        Talker_out = Talker_Filter_list['Flirt_Frequency'].sum() / Talker_Filter_list.shape[0]
        result.append(Talker_out)

    if not Less_Filter_list.empty:
        Less_Filter_list['Flirt_Frequency'] = (Less_Filter_list / len(flirt_words)) * 100
        Less_out = Less_Filter_list['Flirt_Frequency'].sum() / Less_Filter_list.shape[0]
        result.append(Less_out)

    return result


In [7]:
output = flirt_analysis(dataset)


if output:
    talkative_person = dataset['Name'].value_counts().idxmax()
    print("Talkative Person:", talkative_person)
    print("Flirting Percentage of", talkative_person, ":", output[0], "%")

    if len(output) > 1:
        less_talkative_person = dataset['Name'].value_counts().idxmin()
        print("Less Talkative Person:", less_talkative_person)
        print("Flirting Percentage of", less_talkative_person, ":", output[1], "%")
else:
    print("No flirt words found for any person.")

Talkative Person:  AR❤
Flirting Percentage of  AR❤ : 4.166666666666666 %


In [8]:
most_active_date =dataset['Date'].value_counts().idxmax()
print("Most Active Date:", most_active_date)


week_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
date_obj = datetime.datetime.strptime(most_active_date, "%d/%m/%y")
active_day_of_week = week_days[date_obj.weekday()]
print("Active Day of the Week:", active_day_of_week)

# Most active hour of the day
most_active_hour = dataset['Time'].value_counts().idxmax()
print("Most Active Hour of the Day:", most_active_hour)

Most Active Date: 05/12/19
Active Day of the Week: Thursday
Most Active Hour of the Day:  1:43 pm 


In [9]:
media_count = dataset[dataset['Text'] == " <media omitted>"].shape[0]
print("Media Count:", media_count)

# Deleted message count
deleted_message_count =dataset[dataset['Text'] == " this message was deleted"].shape[0]
print("Deleted Message Count:", deleted_message_count)

# Missed voice call count
missed_voice_call_count = dataset[dataset['Text'] == " missed voice call"].shape[0]
print("Missed Voice Call Count:", missed_voice_call_count)

# Missed video call count
missed_video_call_count = dataset[dataset['Text'] == " missed video call"].shape[0]
print("Missed Video Call Count:", missed_video_call_count)

Media Count: 0
Deleted Message Count: 0
Missed Voice Call Count: 0
Missed Video Call Count: 0
